## Develop `wltp/cycler.py`
(WIP) run a vehicle from the h5db, step-by-step.

In [1]:
## To autoreload codein python files here.
%load_ext autoreload
%autoreload 2

## Auto-format cells to ease diffs.
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## If you change that, restart kernel and clear all outpouts before running it
#%matplotlib widget
%matplotlib inline

In [3]:
from typing import Union, List, Callable, Any, Sequence as Seq
import io
import itertools as itt
import logging
from pathlib import Path, PurePosixPath as P
import sys

from columnize import columnize
import numpy as np
import pandas as pd
from pandas import HDFStore, IndexSlice as idx
from pandas.core.generic import NDFrame
from matplotlib import pyplot as plt
from pandalone.mappings import Pstep
import qgrid

from graphtik import operation, compose, optional, sideffec
import wltp
from wltp.experiment import Experiment
from wltp.utils import yaml_loads, pwd_changed
from wltp import datamodel, io as wio, engine, vmax, vehicle, cycler

## Add tests/ into `sys.path` to import `vehdb` module.
#
proj_dir = str(Path(wltp.__file__).parents[1] / "tests")
if proj_dir not in sys.path:
    sys.path.insert(0, proj_dir)

import vehdb

idx = pd.IndexSlice
log = logging.getLogger("VMax.ipynb")
logging.getLogger("blib2to3").propagate = False  # Disable `lab_black` logs.
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s|%(levelname)5.5s|%(name)s:[%(funcName)s]:\n  +--> %(message)s",
    datefmt="%Y-%m-%d,%H:%M:%S",
)
pd.set_option("display.max_columns", 64)

In [4]:
from oct2py import octave

%load_ext oct2py.ipython

In [5]:
octave.addpath(str(Path("Matlab/src").absolute()))

'.:/home/ankostis/Work/wltp.git/.venv/lib/python3.8/site-packages/oct2py:/home/ankostis/Work/wltp.git/.venv/lib/python3.8/site-packages/octave_kernel:/usr/lib/x86_64-linux-gnu/octave/5.2.0/site/oct/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/api-v53/x86_64-pc-linux-gnu:/usr/lib/x86_64-linux-gnu/octave/site/oct/x86_64-pc-linux-gnu:/usr/share/octave/5.2.0/site/m:/usr/share/octave/site/api-v53/m:/usr/share/octave/site/m:/usr/share/octave/site/m/startup:/usr/lib/x86_64-linux-gnu/octave/5.2.0/oct/x86_64-pc-linux-gnu:/usr/share/octave/5.2.0/m:/usr/share/octave/5.2.0/m/audio:/usr/share/octave/5.2.0/m/debian:/usr/share/octave/5.2.0/m/deprecated:/usr/share/octave/5.2.0/m/elfun:/usr/share/octave/5.2.0/m/general:/usr/share/octave/5.2.0/m/geometry:/usr/share/octave/5.2.0/m/gui:/usr/share/octave/5.2.0/m/help:/usr/share/octave/5.2.0/m/image:/usr/share/octave/5.2.0/m/io:/usr/share/octave/5.2.0/m/java:/usr/share/octave/5.2.0/m/legacy:/usr/share/octave/5.2.0/m/linear-algebra:/usr/shar

In [6]:
with pwd_changed("src"):
    octave.calc_all_cases()

read input files :
- case.txt
  ...reading 88 out of 126 in 7.0 sec
- vehicle.txt
- engine.txt
  ...reading 1 out of 5095 in 57.5 sec
  ...reading 1026 out of 5095 in 64.5 sec
  ...reading 2041 out of 5095 in 71.5 sec
  ...reading 3072 out of 5095 in 78.5 sec
  ...reading 4102 out of 5095 in 85.5 sec
  ...reading 5087 out of 5095 in 92.5 sec
- gearbox.txt
  ...reading 613 out of 650 in 7.0 sec
- phase.txt
- trace.txt
  ...reading 1 out of 7015 in 84.0 sec
  ...reading 116 out of 7015 in 91.0 sec
  ...reading 227 out of 7015 in 98.0 sec
  ...reading 333 out of 7015 in 105.0 sec
  ...reading 437 out of 7015 in 112.1 sec
  ...reading 536 out of 7015 in 119.1 sec
  ...reading 629 out of 7015 in 126.2 sec
  ...reading 723 out of 7015 in 133.2 sec
  ...reading 817 out of 7015 in 140.2 sec
  ...reading 912 out of 7015 in 147.3 sec
  ...reading 1002 out of 7015 in 154.3 sec
  ...reading 1094 out of 7015 in 161.4 sec
  ...reading 1178 out of 7015 in 168.4 sec
  ...reading 1260 out of 7015 in 17

Oct2PyError: Octave evaluation error:
error: strfind: each element of CELLSTR must be a string
error: called from:
    calc_all_cases>get_tab_idx at line 1056, column 11
    calc_all_cases>get_tab_row at line 1038, column 9
    calc_all_cases at line 386, column 18